In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.onnx
import onnx
import onnxruntime as ort
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Step 1: Generate the dataset
def generate_dataset(size=10, save_csv=True, print_sample=True):
    # Generate random integers from 1 to 9
    numbers = np.random.randint(1, 10, size)
    # Label them as 1 for odd, 0 for even
    labels = numbers % 2
    
    # Create simplified DataFrame
    df = pd.DataFrame({
        'number': numbers,
        'label': labels
    })
    
    # Save to CSV if requested
    if save_csv:
        csv_filename = 'd:/Desktop/ai/even_odd_dataset.csv'
        df.to_csv(csv_filename, index=False)
        print(f"Dataset saved to: {csv_filename}")
    
    # Print sample of the dataset if requested
    if print_sample:
        print("\n=== Dataset Sample ===")
        print(f"Total dataset size: {len(df)}")
        print("\nDataset preview:")
        print(df.head(10))
        
        print("\n=== Dataset Statistics ===")
        print(f"Even numbers count: {len(df[df['label'] == 0])}")
        print(f"Odd numbers count: {len(df[df['label'] == 1])}")
        print(f"Percentage distribution: {len(df[df['label'] == 0]) / len(df) * 100:.1f}% even, {len(df[df['label'] == 1]) / len(df) * 100:.1f}% odd")
    
    return numbers, labels, df

In [3]:
# Step 2: Preprocess the data
def preprocess_data(numbers, labels):
    # Create multiple features to help the neural network learn better
    features = []
    for num in numbers:
        # Feature 1: Last digit (most important for odd/even)
        last_digit = num % 10
        
        # Feature 2: Is last digit even (0) or odd (1)
        is_odd_digit = last_digit % 2
        
        # Feature 3: Normalized last digit
        normalized_last_digit = last_digit / 9.0
        
        # Use multiple features
        features.append([normalized_last_digit, is_odd_digit, last_digit / 10.0])
    
    # Convert to PyTorch tensors
    features = torch.tensor(features, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32).reshape(-1, 1)
    return features, labels

In [4]:
# Step 3: Build the AI model
class EvenOddModel(nn.Module):
    def __init__(self):
        super(EvenOddModel, self).__init__()
        self.fc1 = nn.Linear(3, 32)  # 3 input features
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

def build_model():
    model = EvenOddModel()
    return model

# Alternative simplified approach - use the number modulo operation directly as feature


In [5]:
# Step 3.5: Visualize the model architecture
def visualize_model(model):
    # Display model summary
    print("Model Summary:")
    print(model)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\nTotal parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")

In [6]:
# Step 4: Train the model
def train_model(model, x_train, y_train, epochs=50, batch_size=64):
    # Set up loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.8)
    
    # Create data loader
    dataset = TensorDataset(x_train, y_train)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0
        
        for batch_x, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
        
        scheduler.step()
        
        if (epoch + 1) % 20 == 0:
            accuracy = 100 * correct / total
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(dataloader):.4f}, Accuracy: {accuracy:.2f}%')
    
    return model

In [7]:
# Step 5: Use the model to predict even/odd
def predict_even_or_odd(model, number):
    # Use the same feature engineering as training
    last_digit = number % 10
    is_odd_digit = last_digit % 2
    normalized_last_digit = last_digit / 9.0
    
    # Create the same features as training
    features = [normalized_last_digit, is_odd_digit, last_digit / 10.0]
    input_tensor = torch.tensor([features], dtype=torch.float32)
    
    model.eval()
    with torch.no_grad():
        prediction = model(input_tensor)
    
    if prediction.item() > 0.5:
        return "odd"
    else:
        return "even"

In [8]:
# Step 5.5: Load dataset from CSV
def load_dataset_from_csv(csv_path='d:/Desktop/ai/even_odd_dataset.csv'):
    try:
        df = pd.read_csv(csv_path)
        print(f"Dataset loaded from: {csv_path}")
        print(f"Dataset shape: {df.shape}")
        print("\nDataset preview:")
        print(df.head())
        
        numbers = df['number'].values
        labels = df['label'].values
        
        return numbers, labels, df
    except FileNotFoundError:
        print(f"CSV file not found at: {csv_path}")
        print("Please generate the dataset first.")
        return None, None, None

## Generate and Save Dataset
Generate a dataset of 1000 numbers and save it to CSV file with detailed statistics.

In [9]:
# Generate and save dataset to CSV
print("=== Generating Dataset ===")
numbers, labels, df = generate_dataset(size=1000, save_csv=True, print_sample=True)

=== Generating Dataset ===
Dataset saved to: d:/Desktop/ai/even_odd_dataset.csv

=== Dataset Sample ===
Total dataset size: 1000

Dataset preview:
   number  label
0       8      0
1       9      1
2       5      1
3       2      0
4       2      0
5       7      1
6       1      1
7       3      1
8       8      0
9       3      1

=== Dataset Statistics ===
Even numbers count: 440
Odd numbers count: 560
Percentage distribution: 44.0% even, 56.0% odd


## Load Dataset from CSV
Load the previously saved dataset from CSV file.

In [10]:
# Load dataset from CSV
print("=== Loading Dataset from CSV ===")
loaded_numbers, loaded_labels, loaded_df = load_dataset_from_csv()

if loaded_numbers is not None:
    print("Dataset loaded successfully!")
else:
    print("Failed to load dataset. Please run the dataset generation cell first.")

=== Loading Dataset from CSV ===
Dataset loaded from: d:/Desktop/ai/even_odd_dataset.csv
Dataset shape: (1000, 2)

Dataset preview:
   number  label
0       8      0
1       9      1
2       5      1
3       2      0
4       2      0
Dataset loaded successfully!


## Preprocess Data and Build Model
Preprocess the loaded data and create the neural network model.

In [11]:
# Preprocess the data
if 'loaded_numbers' in locals() and loaded_numbers is not None:
    x_train, y_train = preprocess_data(loaded_numbers, loaded_labels)
    print(f"Data preprocessed. Training set size: {len(x_train)}")
    
    # Build the model
    model = build_model()
    print("Model built successfully!")
else:
    print("Please load the dataset first.")

Data preprocessed. Training set size: 1000
Model built successfully!


## Train the Model
Train the neural network on the preprocessed data.

In [12]:
# Train the model
if 'model' in locals() and 'x_train' in locals():
    print("=== Training Model ===")
    train_model(model, x_train, y_train, epochs=50)
    print("Model training completed!")
else:
    print("Please build the model and preprocess data first.")

=== Training Model ===
Epoch [20/50], Loss: 0.0067, Accuracy: 100.00%
Epoch [20/50], Loss: 0.0067, Accuracy: 100.00%
Epoch [40/50], Loss: 0.0018, Accuracy: 100.00%
Model training completed!
Epoch [40/50], Loss: 0.0018, Accuracy: 100.00%
Model training completed!


## Test Predictions
Test the trained model on sample numbers to verify accuracy.

In [13]:
# Test predictions
if 'model' in locals():
    print("=== Testing Predictions ===")
    test_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
    for num in test_numbers:
        prediction = predict_even_or_odd(model, num)
        actual = "odd" if num % 2 == 1 else "even"
        status = "✓" if prediction == actual else "✗"
        print(f"Number: {num}, Predicted: {prediction}, Actual: {actual} {status}")
else:
    print("Please train the model first.")

=== Testing Predictions ===
Number: 1, Predicted: odd, Actual: odd ✓
Number: 2, Predicted: even, Actual: even ✓
Number: 3, Predicted: odd, Actual: odd ✓
Number: 4, Predicted: even, Actual: even ✓
Number: 5, Predicted: odd, Actual: odd ✓
Number: 6, Predicted: even, Actual: even ✓
Number: 7, Predicted: odd, Actual: odd ✓
Number: 8, Predicted: even, Actual: even ✓
Number: 9, Predicted: odd, Actual: odd ✓
Number: 0, Predicted: even, Actual: even ✓


In [14]:
# Step 6: Save and Load Model
def save_model(model, filepath='d:/Desktop/ai/even_odd_model.pth'):
    """Save the trained model to file"""
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_architecture': 'EvenOddModel'
    }, filepath)
    print(f"Model saved to: {filepath}")

def export_to_onnx(model, filepath='d:/Desktop/ai/even_odd_model.onnx'):
    """Export the trained model to ONNX format"""
    # Create a dummy input tensor with the same shape as your training data
    dummy_input = torch.randn(1, 3)  # Batch size 1, 3 features
    
    # Export the model
    torch.onnx.export(
        model,                          # model being run
        dummy_input,                    # model input (or a tuple for multiple inputs)
        filepath,                       # where to save the model
        export_params=True,             # store the trained parameter weights inside the model file
        opset_version=11,               # the ONNX version to export the model to
        do_constant_folding=True,       # whether to execute constant folding for optimization
        input_names=['input'],          # the model's input names
        output_names=['output'],        # the model's output names
        dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                     'output': {0: 'batch_size'}}
    )
    print(f"Model exported to ONNX: {filepath}")

def load_onnx_model(filepath='d:/Desktop/ai/even_odd_model.onnx'):
    """Load and create ONNX runtime session"""
    try:
        ort_session = ort.InferenceSession(filepath)
        print(f"ONNX model loaded from: {filepath}")
        return ort_session
    except FileNotFoundError:
        print(f"ONNX model file not found at: {filepath}")
        return None

def predict_with_onnx(ort_session, number):
    """Make prediction using ONNX model"""
    # Use the same feature engineering as training
    last_digit = number % 10
    is_odd_digit = last_digit % 2
    normalized_last_digit = last_digit / 9.0
    
    # Create the same features as training
    features = np.array([[normalized_last_digit, is_odd_digit, last_digit / 10.0]], dtype=np.float32)
    
    # Run inference
    ort_inputs = {ort_session.get_inputs()[0].name: features}
    ort_outputs = ort_session.run(None, ort_inputs)
    prediction = ort_outputs[0][0][0]
    
    if prediction > 0.5:
        return "odd"
    else:
        return "even"

def load_model(filepath='d:/Desktop/ai/even_odd_model.pth'):
    """Load a trained model from file"""
    try:
        checkpoint = torch.load(filepath)
        model = EvenOddModel()
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        print(f"Model loaded from: {filepath}")
        return model
    except FileNotFoundError:
        print(f"Model file not found at: {filepath}")
        return None

## Save Trained Model
Save the trained model to disk for future use.


In [15]:
# Save the trained model
if 'model' in locals():
    print("=== Saving Model ===")
    save_model(model)
    
    # Export to ONNX
    print("=== Exporting to ONNX ===")
    export_to_onnx(model)
    print("Model saved and exported successfully!")
else:
    print("Please train the model first.")

## Load Model
##Load a previously saved model from disk.

# Load a saved model (optional - for demonstration)
print("=== Loading PyTorch Model Demo ===")
loaded_model = load_model()
if loaded_model is not None:
    print("PyTorch model loaded successfully!")
    
    # Test the loaded model
    test_number = 7
    prediction = predict_even_or_odd(loaded_model, test_number)
    actual = "odd" if test_number % 2 == 1 else "even"
    print(f"PyTorch model prediction for {test_number}: {prediction} (actual: {actual})")

# Load and test ONNX model
print("\n=== Loading ONNX Model Demo ===")
onnx_session = load_onnx_model()
if onnx_session is not None:
    print("ONNX model loaded successfully!")
    
    # Test the ONNX model
    test_number = 5
    onnx_prediction = predict_with_onnx(onnx_session, test_number)
    actual = "odd" if test_number % 2 == 1 else "even"
    print(f"ONNX model prediction for {test_number}: {onnx_prediction} (actual: {actual})")

=== Saving Model ===
Model saved to: d:/Desktop/ai/even_odd_model.pth
=== Exporting to ONNX ===
Model exported to ONNX: d:/Desktop/ai/even_odd_model.onnx
Model saved and exported successfully!
=== Loading PyTorch Model Demo ===
Model loaded from: d:/Desktop/ai/even_odd_model.pth
PyTorch model loaded successfully!
PyTorch model prediction for 7: odd (actual: odd)

=== Loading ONNX Model Demo ===
ONNX model loaded from: d:/Desktop/ai/even_odd_model.onnx
ONNX model loaded successfully!
ONNX model prediction for 5: odd (actual: odd)
